# Pré Processamento

Este notebook tem como objetivo preparar os dados para modelagem preditiva de risco de crédito, transformando variáveis categóricas e numéricas em um formato adequado para algoritmos de aprendizado de máquina, garantindo reprodutibilidade, interpretabilidade e ausência de vazamento de dados.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
df = pd.read_csv("../data/processed/german_credit_clean.csv")

df.head()

,status_checking_account,duration_months,credit_history,purpose,credit_amount,savings_account,employment_since,installment_rate,personal_status_sex,other_debtors,...,property,age,other_installment_plans,housing,existing_credits,job,num_dependents,telephone,foreign_worker,target
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,0
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,1
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,0
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,0
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,1


In [3]:
X = df.drop(columns="target")
y = df["target"]

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   status_checking_account  1000 non-null   object
 1   duration_months          1000 non-null   int64 
 2   credit_history           1000 non-null   object
 3   purpose                  1000 non-null   object
 4   credit_amount            1000 non-null   int64 
 5   savings_account          1000 non-null   object
 6   employment_since         1000 non-null   object
 7   installment_rate         1000 non-null   int64 
 8   personal_status_sex      1000 non-null   object
 9   other_debtors            1000 non-null   object
 10  residence_since          1000 non-null   int64 
 11  property                 1000 non-null   object
 12  age                      1000 non-null   int64 
 13  other_installment_plans  1000 non-null   object
 14  housing                  1000 non-null   

In [5]:
variaveis_numericas = X.select_dtypes(include=["int64"]).columns.tolist()
variaveis_categoricas = X.select_dtypes(include=["object"]).columns.tolist()

print("Variáveis numéricas:")
print(variaveis_numericas)
print()

print("Variáveis categóricas:")
print(variaveis_categoricas)

Variáveis numéricas:
['duration_months', 'credit_amount', 'installment_rate', 'residence_since', 'age', 'existing_credits', 'num_dependents']

Variáveis categóricas:
['status_checking_account', 'credit_history', 'purpose', 'savings_account', 'employment_since', 'personal_status_sex', 'other_debtors', 'property', 'other_installment_plans', 'housing', 'job', 'telephone', 'foreign_worker']


In [6]:
preprocessador = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), variaveis_numericas),
        ("cat", OneHotEncoder(handle_unknown="ignore", drop="first"), variaveis_categoricas)
    ]
)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

In [9]:
X_train_fe = preprocessador.fit_transform(X_train)
X_test_fe = preprocessador.fit_transform(X_test)

In [10]:
print("Treino:", X_train_fe.shape)
print("Teste:", X_test_fe.shape)

Treino: (700, 48)
Teste: (300, 48)


A separação entre os conjuntos de treino e teste foi realizada antes da aplicação das transformações, evitando vazamento de dados. O uso do ColumnTransformer garante que o mesmo pré-processamento seja aplicado de forma consistente tanto nos dados de treinamento quanto em novos dados.

Após a etapa de feature engineering, o conjunto de treino ficou com 700 observações e 48 variáveis, enquanto o conjunto de teste possui 300 observações e as mesmas 48 variáveis. O aumento no número de features é esperado devido à aplicação do One-Hot Encoding nas variáveis categóricas.